In [116]:
import pickle
import numpy as np
import pandas as pd

In [117]:
BOFevent = np.array( pickle.load(open('labeled_Data/labelsBOF.p', 'rb')) ).astype(int)
BOFCarpool=np.array( pickle.load(open('labeled_Data/labelsBOFCarpool.p','rb')) ).astype(int)

Converting all the True/Falses into 1's and 0's. From here we will multiply the carpool event by 2 in order to change it's label and add the two together to have 1 array that's to be one-hot encoded later.

In [118]:
Y = BOFCarpool*2+BOFevent
Y_one_hot = np.zeros((len(Y), len(set(Y))))

In [124]:
for index, y in enumerate(Y):
    Y_one_hot[index, y] = 1

In [127]:
print(Y_one_hot, '\n')
print(Y)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]] 

[0 1 2 ... 0 1 1]


Next we'll take the events that the Y's are associated with and convert them to a bag of words model. We'll use this model because the order is not necessary.

In [63]:
events=np.array(pd.read_pickle( 'labeled_Data/cleanedEvents.p'))

In [130]:
dictionary = {'UNK':0}

In [131]:
events[0].split()

['Three', "King's", 'Day', 'Party']

In [132]:
counter = 1
for event in events:
    for word in event.lower().split():
        if word not in dictionary:
            dictionary[word] = counter
            counter+=1

In [133]:
len(dictionary) #number of tokens in our dictionary

1855

In [134]:
len(events) #number of data points we have

1163

In [135]:
bag_of_words = np.zeros((len(events), len(dictionary))).astype(int)

In [136]:
for index, event in enumerate(events):
    for word in event.lower().split():
        word_index = dictionary[word]
        bag_of_words[index, word_index] +=1

We now have our bag of words. We can construct a simple neural network to determine whether an event is irrelevant, BOF or BOFCarpool.

In [137]:
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

In [138]:
X_train, X_test, Y_train, Y_test = train_test_split(bag_of_words, Y_one_hot, test_size = 0.1)

In [197]:
model = Sequential()
model.add(Dense(500, input_shape = (len(dictionary), ) ))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(Y_one_hot.shape[1]))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [198]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 500)               928000    
_________________________________________________________________
activation_37 (Activation)   (None, 500)               0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 500)               250500    
_________________________________________________________________
activation_38 (Activation)   (None, 500)               0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 3)                 1503      
__________

In [199]:
fit = model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
1046/1046 [==============================] - 2s 2ms/step - loss: 0.5001 - acc: 0.8719
Epoch 2/10
1046/1046 [==============================] - 1s 792us/step - loss: 0.2459 - acc: 0.9207
Epoch 3/10
1046/1046 [==============================] - 1s 747us/step - loss: 0.1610 - acc: 0.9207
Epoch 4/10
1046/1046 [==============================] - 1s 754us/step - loss: 0.1258 - acc: 0.9293
Epoch 5/10
1046/1046 [==============================] - 1s 757us/step - loss: 0.0916 - acc: 0.9618
Epoch 6/10
1046/1046 [==============================] - 1s 761us/step - loss: 0.0646 - acc: 0.9857
Epoch 7/10
1046/1046 [==============================] - 1s 961us/step - loss: 0.0424 - acc: 0.9933
Epoch 8/10
1046/1046 [==============================] - 1s 843us/step - loss: 0.0293 - acc: 0.9933
Epoch 9/10
1046/1046 [==============================] - 1s 803us/step - loss: 0.0264 - acc: 0.9924
Epoch 10/10
1046/1046 [==============================] - 1s 954us/step - loss: 0.0222 - acc: 0.9952


In [200]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.04167985597736815
Test accuracy: 0.9914529919624329


For a quick neural network, the best I was able to get was 99.1%. Not perfect but pretty close!